# Terraform on GCP - Implementation Steps

This notebook contains all the implementation steps for deploying infrastructure on Google Cloud Platform using Terraform. Each step includes code snippets, commands, and detailed descriptions.

## Table of Contents

1. **Phase 1: Infrastructure Setup with gcloud CLI**
   - Task 1: Containerize the Application
   - Task 2: Set up Service Account
   - Task 3: Configure Authentication

2. **Phase 2: Infrastructure as Code with Terraform**
   - Task 4: Configure Variables
   - Task 5: Configure Outputs
   - Task 6: Define Provider
   - Task 7: Initialize Infrastructure
   - Task 8: Create Storage Bucket
   - Task 9: Store Terraform State in Backend
   - Task 10: Set up GKE Cluster
   - Task 11: Deploy Worker Nodes
   - Task 12: Set up Connection with Worker Nodes

3. **Phase 3: Deploy Application to Cloud Run**
   - Task 13: Enable and Create Artifact Registry
   - Task 14: Push Image to Artifact Registry
   - Task 15: Enable Cloud Run API
   - Task 16: Host Container on Cloud Run
   - Task 17: Locate the Address of Deployed Service

4. **Cleanup**
   - Task 18: Destroy All Resources

## Phase 1: Infrastructure Setup with gcloud CLI

### Task 1: Containerize the Application

**Description**: Build a Docker image from the Dockerfile and push it to Docker Hub.

**Prerequisites**:
- Docker Desktop installed and running
- Docker Hub account created

#### Option 1: Using the automated script (Recommended)

Navigate to the Application directory and run the build-and-push script:

In [ ]:
# Navigate to Application directory
cd Application

# Run the build-and-push script
./build-and-push.sh YOUR_DOCKERHUB_USERNAME

#### Option 2: Manual steps

If you prefer to run the commands manually:

In [ ]:
# Navigate to Application directory
cd Application

# Build the Docker image
docker build -t terraform-gcp-app:latest .

# Tag the image for Docker Hub
docker tag terraform-gcp-app:latest YOUR_DOCKERHUB_USERNAME/terraform-gcp-app:latest

# Login to Docker Hub
docker login

# Push the image to Docker Hub
docker push YOUR_DOCKERHUB_USERNAME/terraform-gcp-app:latest

### Task 2: Set up Service Account

**Description**: Create a GCP project in your billing account and set up a service account for Terraform operations.

**Prerequisites**:
- GCP account with billing enabled
- gcloud CLI installed and authenticated

#### Step 1: Create a project in the billing account

In [ ]:
# List available billing accounts (if needed)
gcloud billing accounts list

# Create a new GCP project
gcloud projects create PROJECT_ID \
  --name="PROJECT_NAME" \
  --set-as-default

# Link the project to your billing account
gcloud billing projects link PROJECT_ID \
  --billing-account=BILLING_ACCOUNT_ID

# Set the project as the default
gcloud config set project PROJECT_ID

# Enable required APIs
gcloud services enable compute.googleapis.com
gcloud services enable container.googleapis.com
gcloud services enable storage-api.googleapis.com
gcloud services enable artifactregistry.googleapis.com
gcloud services enable run.googleapis.com

#### Step 2: Create a service account

In [ ]:
# Create the service account
gcloud iam service-accounts create terraform-sa \
  --display-name="Terraform Service Account" \
  --description="Service account for Terraform infrastructure management"

# Grant necessary IAM roles to the service account
gcloud projects add-iam-policy-binding PROJECT_ID \
  --member="serviceAccount:terraform-sa@PROJECT_ID.iam.gserviceaccount.com" \
  --role="roles/editor"

gcloud projects add-iam-policy-binding PROJECT_ID \
  --member="serviceAccount:terraform-sa@PROJECT_ID.iam.gserviceaccount.com" \
  --role="roles/storage.admin"

gcloud projects add-iam-policy-binding PROJECT_ID \
  --member="serviceAccount:terraform-sa@PROJECT_ID.iam.gserviceaccount.com" \
  --role="roles/container.admin"

gcloud projects add-iam-policy-binding PROJECT_ID \
  --member="serviceAccount:terraform-sa@PROJECT_ID.iam.gserviceaccount.com" \
  --role="roles/run.admin"

# Create and download service account key
gcloud iam service-accounts keys create account.json \
  --iam-account=terraform-sa@PROJECT_ID.iam.gserviceaccount.com

# Set the credentials environment variable
export GOOGLE_APPLICATION_CREDENTIALS="$(pwd)/account.json"

### Task 3: Configure Authentication

**Description**: Create a service account key pair and ensure appropriate policy bindings are configured.

**Note**: If you completed Task 2, you may have already created the key. This task focuses on authentication setup.

In [ ]:
# Create service account key (if not already created)
gcloud iam service-accounts keys create account.json \
  --iam-account=terraform-sa@PROJECT_ID.iam.gserviceaccount.com

# Set up application default credentials
export GOOGLE_APPLICATION_CREDENTIALS="$(pwd)/account.json"

# Authenticate with gcloud CLI
gcloud auth activate-service-account terraform-sa@PROJECT_ID.iam.gserviceaccount.com \
  --key-file=account.json

# Verify authentication
gcloud auth list
gcloud projects list

## Phase 2: Infrastructure as Code with Terraform

### Task 4: Configure Variables

**Description**: Configure Terraform variables to make configurations reusable and environment-aware.

**File**: `variables.tf`

#### Review and update variables.tf

The variables.tf file should contain:

In [ ]:
# variables.tf content
variable "project_id" {
  type    = string
  default = "YOUR_PROJECT_ID"
}

variable "region" {
  type    = string
  default = "us-east1"
}

variable "state_bucket" {
  type    = string
  default = "terraform-state-YOUR-PROJECT-ID"
}

variable "cluster_name" {
  type    = string
  default = "poc-terraform"
}

variable "service_name" {
  type    = string
  default = "poc-terraform-sample"
}

variable "k8s_version" {
  type    = string
  default = "1.24"
}

variable "min_node_count" {
  type    = number
  default = 1
}

variable "max_node_count" {
  type    = number
  default = 3
}

variable "machine_type" {
  type    = string
  default = "e2-standard-2"
}

variable "preemptible" {
  type    = bool
  default = true
}

variable "artifact_registry_repository" {
  type    = string
  default = "docker-repo"
}

variable "cloud_run_image_name" {
  type    = string
  default = "terraform-gcp-app"
}

variable "cloud_run_image_tag" {
  type    = string
  default = "latest"
}

#### Validate variable configuration

In [ ]:
# Validate Terraform syntax
terraform validate

# Preview changes with current variable values
terraform plan

### Task 5: Configure Outputs

**Description**: Configure Terraform outputs to expose important information about your infrastructure.

**File**: `output.tf`

In [ ]:
# output.tf content
output "project_id" {
  description = "The name of the project"
  value       = var.project_id
}

output "state_bucket" {
  description = "The name of the storage bucket that will store the state of the infrastructure"
  value       = var.state_bucket
}

output "cluster_name" {
  description = "The name of the deployed cluster"
  value       = var.cluster_name
}

output "k8s_version" {
  description = "The Kubernetes version used to deploy to the cluster"
  value       = var.k8s_version
}

output "region" {
  description = "The region where all the infrastructure is deployed"
  value       = var.region
}

output "cloud_run_url" {
  description = "The URL of the deployed Cloud Run service"
  value       = google_cloud_run_service.app.status[0].url
}

### Task 6: Define Provider

**Description**: Configure the Google Cloud Platform provider for Terraform.

**File**: `main.tf` (provider block)

In [ ]:
# Provider configuration in main.tf
provider "google" {
  credentials = file("account.json")
  project     = var.project_id
  region      = var.region
}

### Task 7: Initialize Infrastructure

**Description**: Initialize Terraform working directory and apply infrastructure changes.

**Commands**: `terraform init`, `terraform plan`, `terraform apply`

In [ ]:
# Initialize Terraform (downloads provider plugins)
terraform init

# Format Terraform files
terraform fmt

# Validate configuration syntax
terraform validate

# Preview changes before applying
terraform plan

# Apply infrastructure changes
terraform apply

### Task 8: Create Storage Bucket

**Description**: Create a Google Cloud Storage bucket for storing Terraform state.

**File**: `main.tf` (storage bucket resource)

In [ ]:
# Storage bucket resource in main.tf
resource "google_storage_bucket" "state" {
  name          = var.state_bucket
  location      = var.region
  project       = var.project_id
  storage_class = "STANDARD"
  force_destroy = true
  versioning {
    enabled = true
  }
}

#### Verify the bucket

In [ ]:
# List buckets in your project
gsutil ls

# Get bucket details
gsutil ls -L gs://BUCKET_NAME

### Task 9: Store Terraform State in Backend

**Description**: Configure Terraform to store its state file in the Google Cloud Storage bucket.

**File**: `main.tf` (terraform backend block)

In [ ]:
# Backend configuration in main.tf
terraform {
  backend "gcs" {
    bucket      = "terraform-state-YOUR-PROJECT-ID"
    prefix      = "terraform/state"
    credentials = "account.json"
  }
}

#### Initialize with backend configuration

In [ ]:
# Initialize Terraform with backend
# Terraform will prompt to migrate existing state - answer 'yes'
terraform init

# Verify state is stored in backend
gsutil ls gs://BUCKET_NAME/terraform/state/

### Task 10: Set up GKE Cluster

**Description**: Create a Google Kubernetes Engine (GKE) cluster using Terraform.

**Prerequisites**: Kubernetes Engine API must be enabled

In [ ]:
# Enable Kubernetes Engine API
gcloud services enable container.googleapis.com --project=PROJECT_ID

# Find valid Kubernetes versions
gcloud container get-server-config --region=REGION --project=PROJECT_ID

#### GKE Cluster resource in main.tf

In [ ]:
# GKE cluster resource in main.tf
resource "google_container_cluster" "primary" {
  name     = var.cluster_name
  location = var.region
  project  = var.project_id

  # Remove default node pool, we'll create a separate one
  remove_default_node_pool = true
  initial_node_count       = 1

  # Disable deletion protection to allow cluster deletion
  deletion_protection = false

  # Network configuration
  network    = "default"
  subnetwork = "default"

  # Enable logging and monitoring
  logging_service    = "logging.googleapis.com/kubernetes"
  monitoring_service = "monitoring.googleapis.com/kubernetes"

  # Resource labels
  resource_labels = {
    environment = "development"
    managed-by  = "terraform"
  }
}

#### Verify the cluster

In [ ]:
# List clusters in your project
gcloud container clusters list --project=PROJECT_ID

# Get cluster details
gcloud container clusters describe CLUSTER_NAME --region=REGION --project=PROJECT_ID

### Task 11: Deploy Worker Nodes

**Description**: Create a node pool for your GKE cluster with worker nodes.

**File**: `main.tf` (node pool resource)

In [ ]:
# Node pool resource in main.tf
resource "google_container_node_pool" "primary_nodes" {
  name       = "${var.cluster_name}-node-pool"
  location   = var.region
  cluster    = google_container_cluster.primary.name
  project    = var.project_id
  node_count = var.min_node_count

  # Node configuration subblock
  node_config {
    machine_type = var.machine_type
    disk_size_gb = 100
    disk_type    = "pd-standard"
    preemptible  = var.preemptible

    # OAuth scopes
    oauth_scopes = [
      "https://www.googleapis.com/auth/cloud-platform"
    ]

    # Labels for nodes
    labels = {
      environment = "development"
      managed-by  = "terraform"
    }

    # Metadata
    metadata = {
      disable-legacy-endpoints = "true"
    }

    # Tags
    tags = ["gke-node", "${var.cluster_name}-node"]
  }

  # Management subblock
  management {
    auto_repair  = true
    auto_upgrade = true
  }

  # Autoscaling subblock
  autoscaling {
    min_node_count = var.min_node_count
    max_node_count = var.max_node_count
  }
}

#### Verify the node pool

In [ ]:
# List node pools in the cluster
gcloud container node-pools list \
  --cluster=CLUSTER_NAME \
  --region=REGION \
  --project=PROJECT_ID

# Get cluster credentials
gcloud container clusters get-credentials CLUSTER_NAME \
  --region=REGION \
  --project=PROJECT_ID

# List nodes in the cluster
kubectl get nodes

### Task 12: Set up Connection with Worker Nodes

**Description**: Set up kubectl to connect to your GKE cluster and verify the connection.

**Prerequisites**: kubectl and gke-gcloud-auth-plugin must be installed

In [ ]:
# Set KUBECONFIG environment variable
export KUBECONFIG=~/.kube/config

# Connect to the cluster
gcloud container clusters get-credentials CLUSTER_NAME \
  --region=REGION \
  --project=PROJECT_ID

# Create admin role bindings
GCLOUD_USER=$(gcloud config get-value account)
kubectl create clusterrolebinding cluster-admin-binding \
  --clusterrole=cluster-admin \
  --user=$GCLOUD_USER

# Verify connection and count nodes
kubectl cluster-info
kubectl get nodes

# Count nodes programmatically
kubectl get nodes --no-headers | wc -l

## Phase 3: Deploy Application to Cloud Run

### Task 13: Enable and Create Artifact Registry

**Description**: Create an Artifact Registry repository for storing Docker container images.

**File**: `main.tf` (Artifact Registry resources)

In [ ]:
# Enable Artifact Registry API
gcloud services enable artifactregistry.googleapis.com --project=PROJECT_ID

#### Artifact Registry resources in main.tf

In [ ]:
# Data source to get default compute service account
data "google_compute_default_service_account" "default" {
  project = var.project_id
}

# Artifact Registry Repository
resource "google_artifact_registry_repository" "docker_repo" {
  location      = var.region
  repository_id = var.artifact_registry_repository
  description   = "Docker repository for container images"
  format        = "DOCKER"
  project       = var.project_id
}

# IAM Policy Binding for Artifact Registry - Writer role for GKE nodes
resource "google_artifact_registry_repository_iam_binding" "docker_repo_iam_writer" {
  location   = var.region
  repository = google_artifact_registry_repository.docker_repo.repository_id
  role       = "roles/artifactregistry.writer"
  project    = var.project_id
  members = [
    "serviceAccount:${data.google_compute_default_service_account.default.email}",
  ]
}

# IAM Policy Binding for Artifact Registry - Reader role for service account
resource "google_artifact_registry_repository_iam_binding" "docker_repo_iam_reader" {
  location   = var.region
  repository = google_artifact_registry_repository.docker_repo.repository_id
  role       = "roles/artifactregistry.reader"
  project    = var.project_id
  members = [
    "serviceAccount:educative-service-account@${var.project_id}.iam.gserviceaccount.com",
  ]
}

#### Verify the repository

In [ ]:
# List repositories
gcloud artifacts repositories list --project=PROJECT_ID --location=REGION

# Describe repository
gcloud artifacts repositories describe REPOSITORY_NAME \
  --location=REGION \
  --project=PROJECT_ID

### Task 14: Push Image to Artifact Registry

**Description**: Push a Docker image from Docker Hub to Google Artifact Registry.

**Prerequisites**:
- Docker installed and running
- Image available in Docker Hub (from Task 1)
- Artifact Registry repository created (from Task 13)

In [ ]:
# Authenticate with gcloud
gcloud auth login
gcloud config set account YOUR_EMAIL@gmail.com

# Configure Docker for Artifact Registry authentication
gcloud auth configure-docker REGION-docker.pkg.dev

# Example for us-east1
gcloud auth configure-docker us-east1-docker.pkg.dev

# Pull the image from Docker Hub
docker pull DOCKERHUB_USERNAME/terraform-gcp-app:latest

# Tag the image for Artifact Registry
# Format: REGION-docker.pkg.dev/PROJECT_ID/REPOSITORY_NAME/IMAGE_NAME:TAG
docker tag DOCKERHUB_USERNAME/terraform-gcp-app:latest \
  REGION-docker.pkg.dev/PROJECT_ID/REPOSITORY_NAME/terraform-gcp-app:latest

# Example
docker tag your-username/terraform-gcp-app:latest \
  us-east1-docker.pkg.dev/dolu-sandbox-408209/docker-repo/terraform-gcp-app:latest

# Push the image to Artifact Registry
docker push REGION-docker.pkg.dev/PROJECT_ID/REPOSITORY_NAME/terraform-gcp-app:latest

# Example
docker push us-east1-docker.pkg.dev/dolu-sandbox-408209/docker-repo/terraform-gcp-app:latest

#### Verify the image was pushed

In [ ]:
# List images in the repository
gcloud artifacts docker images list \
  REGION-docker.pkg.dev/PROJECT_ID/REPOSITORY_NAME

# Get image details
gcloud artifacts docker images describe \
  REGION-docker.pkg.dev/PROJECT_ID/REPOSITORY_NAME/terraform-gcp-app:latest

### Task 15: Enable Cloud Run API

**Description**: Enable the Cloud Run API for your GCP project using Terraform.

**File**: `main.tf` (Cloud Run API resource)

In [ ]:
# Enable Cloud Run API resource in main.tf
resource "google_project_service" "cloud_run_api" {
  project = var.project_id
  service = "run.googleapis.com"

  disable_on_destroy = false
}

#### Verify the API is enabled

In [ ]:
# List enabled APIs
gcloud services list --enabled --project=PROJECT_ID | grep run

# Check specific API status
gcloud services describe run.googleapis.com --project=PROJECT_ID

### Task 16: Host Container on Cloud Run

**Description**: Deploy your containerized application to Google Cloud Run using Terraform.

**Prerequisites**:
- Cloud Run API must be enabled (Task 15)
- Docker image must be pushed to Artifact Registry (Task 14)

#### Cloud Run service resources in main.tf

In [ ]:
# Fetch IAM policy for Cloud Run service
data "google_iam_policy" "cloud_run_public_access" {
  binding {
    role = "roles/run.invoker"
    members = [
      "allUsers",
    ]
  }
}

# Cloud Run Service
resource "google_cloud_run_service" "app" {
  name     = var.service_name
  location = var.region
  project  = var.project_id

  template {
    spec {
      containers {
        image = "${var.region}-docker.pkg.dev/${var.project_id}/${var.artifact_registry_repository}/${var.cloud_run_image_name}:${var.cloud_run_image_tag}"

        ports {
          container_port = 8080
        }

        resources {
          limits = {
            cpu    = "1000m"
            memory = "512Mi"
          }
        }
      }
    }
  }

  traffic {
    percent         = 100
    latest_revision = true
  }

  depends_on = [google_project_service.cloud_run_api]
}

# Assign public access to Cloud Run service
resource "google_cloud_run_service_iam_policy" "app_iam_policy" {
  location    = google_cloud_run_service.app.location
  project     = google_cloud_run_service.app.project
  service     = google_cloud_run_service.app.name
  policy_data = data.google_iam_policy.cloud_run_public_access.policy_data
}

#### Verify the Cloud Run service

In [ ]:
# List Cloud Run services
gcloud run services list --region=REGION --project=PROJECT_ID

# Get service details
gcloud run services describe SERVICE_NAME --region=REGION --project=PROJECT_ID

# Get service URL
gcloud run services describe SERVICE_NAME --region=REGION --project=PROJECT_ID --format="value(status.url)"

# Test the service
curl $(gcloud run services describe SERVICE_NAME --region=REGION --project=PROJECT_ID --format="value(status.url)")

### Task 17: Locate the Address of Deployed Service

**Description**: Add an output block to retrieve the URL of your deployed Cloud Run service.

**File**: `output.tf` (already included in Task 5)

#### View the Cloud Run URL output

In [ ]:
# View all outputs
terraform output

# View specific output
terraform output cloud_run_url

# View output as JSON
terraform output -json cloud_run_url

# View output without quotes (raw value)
terraform output -raw cloud_run_url

# Test the service URL
SERVICE_URL=$(terraform output -raw cloud_run_url)
curl $SERVICE_URL

## Cleanup

### Task 18: Destroy All Resources

**Description**: Destroy all infrastructure resources created by Terraform.

**CRITICAL WARNINGS**:

⚠️ **DO NOT DELETE YOUR GCP PROJECT!**

- **Only destroy Terraform-managed resources** using `terraform destroy`
- **Keep your GCP project** - it costs nothing when no resources are running
- Deleting the project will delete **ALL** resources, including those not managed by Terraform
- Project deletion is difficult to reverse and can cause data loss
- If you delete the project, you'll need to create a new one and reconfigure everything

**What to do:**
1. Use `terraform destroy` to remove only Terraform-managed infrastructure
2. Keep the project - you may need it for future work
3. Verify no charges in your billing dashboard
4. Optionally disable billing if you want to ensure no charges (but keep the project)

**What NOT to do:**
- ❌ Do NOT run `gcloud projects delete PROJECT_ID`
- ❌ Do NOT delete the project via the GCP Console
- ❌ Do NOT delete the project thinking it will save money (projects are free)

**Warning**: `terraform destroy` will delete all Terraform-managed resources. Always verify before proceeding.

In [ ]:
# Preview what will be destroyed
terraform plan -destroy

# Destroy all Terraform-managed resources
# NOTE: This will NOT delete your GCP project, only the resources
terraform destroy

# Verify resources are destroyed (but project should still exist)
gcloud projects describe PROJECT_ID  # Should show project exists
gcloud container clusters list --project=PROJECT_ID  # Should be empty
gcloud run services list --region=REGION --project=PROJECT_ID  # Should be empty
gcloud artifacts repositories list --location=REGION --project=PROJECT_ID  # Should be empty

# Verify project still exists and is safe
echo "Project PROJECT_ID should still exist and be active"
gcloud projects describe PROJECT_ID --format="value(projectId,lifecycleState)"

# DO NOT RUN THIS COMMAND - IT WILL DELETE YOUR PROJECT!
# gcloud projects delete PROJECT_ID  # NEVER RUN THIS!